# notebooks/analysis.ipynb

# %% [markdown]
# # Emergency Response Routing - Analysis Notebook
# 
# This notebook analyzes the results of our emergency response routing simulations, comparing Dijkstra's algorithm with A* search under different traffic conditions.

# %%
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

# Set plot style
plt.style.use('seaborn-whitegrid')
sns.set_palette("deep")

# %% [markdown]
# ## Load Simulation Results

# %%
# Define scenarios
scenarios = ['normal', 'rush_hour', 'incident', 'missing_data']

# Load results for each scenario
results = {}
for scenario in scenarios:
    file_path = f"../data/processed/{scenario}_results.csv"
    if os.path.exists(file_path):
        results[scenario] = pd.read_csv(file_path)
        print(f"Loaded {scenario} results: {len(results[scenario])} scenarios")

# Load summary report
summary_path = "../data/processed/summary_report.csv"
if os.path.exists(summary_path):
    summary = pd.read_csv(summary_path)
    print("Loaded summary report")

# %% [markdown]
# ## Analyze Algorithm Performance

# %%
# Create comparison dataframe
performance_comparison = pd.DataFrame({
    'Scenario': [],
    'Algorithm': [],
    'Travel Time (s)': [],
    'Computation Time (s)': []
})

for scenario, df in results.items():
    # Add Dijkstra data
    dijkstra_data = pd.DataFrame({
        'Scenario': [scenario] * len(df),
        'Algorithm': ['Dijkstra'] * len(df),
        'Travel Time (s)': df['dijkstra_travel_time'],
        'Computation Time (s)': df['dijkstra_computation_time']
    })
    
    # Add A* data
    astar_data = pd.DataFrame({
        'Scenario': [scenario] * len(df),
        'Algorithm': ['A*'] * len(df),
        'Travel Time (s)': df['astar_travel_time'],
        'Computation Time (s)': df['astar_computation_time']
    })
    
    # Append to comparison dataframe
    performance_comparison = pd.concat([performance_comparison, dijkstra_data, astar_data])

# %% [markdown]
# ## Visualize Travel Time Comparison

# %%
plt.figure(figsize=(12, 8))
sns.barplot(
    data=performance_comparison,
    x='Scenario',
    y='Travel Time (s)',
    hue='Algorithm'
)
plt.title('Average Travel Time by Algorithm and Scenario', fontsize=16)
plt.xlabel('Traffic Scenario', fontsize=14)
plt.ylabel('Travel Time (seconds)', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Algorithm')
plt.tight_layout()
plt.savefig("../data/processed/travel_time_comparison.png", dpi=300)
plt.show()

# %% [markdown]
# ## Visualize Computation Time Comparison

# %%
plt.figure(figsize=(12, 8))
sns.barplot(
    data=performance_comparison,
    x='Scenario',
    y='Computation Time (s)',
    hue='Algorithm'
)
plt.title('Average Computation Time by Algorithm and Scenario', fontsize=16)
plt.xlabel('Traffic Scenario', fontsize=14)
plt.ylabel('Computation Time (seconds)', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Algorithm')
plt.tight_layout()
plt.savefig("../data/processed/computation_time_comparison.png", dpi=300)
plt.show()

# %% [markdown]
# ## Performance Improvement Analysis

# %%
# Create a summary dataframe with improvement percentages
if 'summary' in locals():
    plt.figure(figsize=(10, 6))
    sns.barplot(
        data=summary,
        x='scenario',
        y='astar_improvement'
    )
    plt.title('A* Improvement Over Dijkstra by Scenario (%)', fontsize=16)
    plt.xlabel('Traffic Scenario', fontsize=14)
    plt.ylabel('Improvement (%)', fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig("../data/processed/improvement_percentage.png", dpi=300)
    plt.show()

# %% [markdown]
# ## Path Similarity Analysis

# %%
if 'summary' in locals():
    plt.figure(figsize=(10, 6))
    sns.barplot(
        data=summary,
        x='scenario',
        y='same_path_percentage'
    )
    plt.title('Percentage of Identical Paths Between Algorithms', fontsize=16)
    plt.xlabel('Traffic Scenario', fontsize=14)
    plt.ylabel('Same Path (%)', fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig("../data/processed/same_path_percentage.png", dpi=300)
    plt.show()

# %% [markdown]
# ## Detailed Scenario Analysis
# 
# Let's look at the distribution of travel times for each scenario.

# %%
for scenario, df in results.items():
    plt.figure(figsize=(12, 6))
    
    # Calculate travel time difference
    df['time_difference'] = df['dijkstra_travel_time'] - df['astar_travel_time']
    
    # Plot histogram of time differences
    sns.histplot(df['time_difference'], kde=True)
    plt.axvline(x=0, color='red', linestyle='--')
    
    plt.title(f'Travel Time Difference (Dijkstra - A*) in {scenario.title()} Scenario', fontsize=16)
    plt.xlabel('Time Difference (seconds)', fontsize=14)
    plt.ylabel('Frequency', fontsize=14)
    plt.tight_layout()
    plt.savefig(f"../data/processed/{scenario}_time_difference.png", dpi=300)
    plt.show()